### Figure 2 : Ripple oscillations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from subjects import sns_boxplot_kw, stat_kw, sns_violin_kw
import pandas as pd
import seaborn as sns
from neuropy.plotting import Fig
from scipy import stats

from statannotations.Annotator import Annotator
from plotters import violinplot, violinplotx


grpdata = subjects.GroupData()
group = ["NSD", "SD"]

# ripple
psd = grpdata.ripple_psd
examples = grpdata.ripple_examples

rpl_features = grpdata.ripple_features
# rpl_features = rpl_features[rpl_features.zt!='PRE'].reset_index(drop=True)
ripple_autocorr = grpdata.ripple_autocorr
# ripple_autocorr = ripple_autocorr[ripple_autocorr.lag_time>=-1]

rate = grpdata.ripple_rate
pbe_rate = grpdata.pbe_rate

# frate
frate_in_rpl = grpdata.frate_in_ripple


plt.clf()
fig = Fig(9, 12, fontsize=5, axis_lw=0.8, tick_size=2, constrained_layout=False)
filename = subjects.figpath_sd / "figure2"

In [ ]:
df = rpl_features.groupby(["grp", "zt"], sort=False).describe()["ripple_power"]
df["iqr"] = df["75%"] - df["25%"]
df

### Peak frequency and power

In [ ]:
# power['zscore'] = np.log10(power['zscore'])
y_labels = [
    "Ripple frequency (Hz)",
    "Ripple power (zscore) \n(Normalized to PRE)",
    "Sharp wave amplitude \n(Normalized to PRE)",
]
gs_ = fig.subplot2grid(
    fig.gs[2:5, :4], grid=(3, 3), width_ratios=[1.5, 1.5, 1.2], hspace=0.4, wspace=0.3
)


for d, y in enumerate(["peak_freq", "ripple_power", "sw_amp"]):
    print(y)
    for g, grp in enumerate(["NSD", "SD"]):
        ax = fig.subplot(gs_[d, g])
        plot_kw = dict(data=rpl_features[rpl_features.grp == grp], x="zt", y=y, ax=ax)

        violinplotx(**plot_kw, color=subjects.colors_sd()[g])
        ax.tick_params("x", labelrotation=30)
        ax.legend([], frameon=False)
        ax.set_ylabel(y_labels[d])
        ax.set_xlabel("")

        # if d > 0:
        #     ax.set_ylim(top=3.2)
        #     ax.set_yticks([0, 1, 2, 3])

        orders = plot_kw["data"].zt.unique()
        pairs = [(orders[_], (orders[_ + 1])) for _ in range(len(orders) - 1)] + [
            ("0-2.5", "5-7.5"),
        ]

        # ax = ax.inset_axes([0,0.8,1,0.3])
        # plot_kw['ax'] = ax
        annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
        annotator.configure(test="t-test_welch", **stat_kw)
        annotator.apply_and_annotate()
        annotator.reset_configuration()

        if d == 1:
            ax.set_ylim(top=15)

        if d == 2:
            ax.set_ylim(top=15)

    ax = fig.subplot(gs_[d, 2])
    df = rpl_features[rpl_features.zt.isin(["0-2.5", "2.5-5", "5-7.5"])].copy()
    df.drop(df[(df.zt == "0-2.5") & (df.grp == "SD")].index, inplace=True)
    df.drop(df[(df.zt == "5-7.5") & (df.grp == "NSD")].index, inplace=True)
    df.loc[(df.zt == "0-2.5") & (df.grp == "NSD"), "zt"] = "0-2.5 vs 5-7.5"
    df.loc[(df.zt == "5-7.5") & (df.grp == "SD"), "zt"] = "0-2.5 vs 5-7.5"
    plot_kw = dict(
        data=df,
        x="zt",
        y=y,
        hue="grp",
        hue_order=["NSD", "SD"],
        ax=ax,
    )

    violinplot(**plot_kw)
    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.legend("", frameon=False)

    if d == 1:
        ax.set_ylim(top=15)

    if d == 2:
        ax.set_ylim(top=15)

    # if d > 0:
    #     ax.set_ylim(top=3.2)
    #     ax.set_yticks([0, 1, 2, 3])

    orders = plot_kw["data"].zt.unique()
    pairs = [
        (("2.5-5", "NSD"), ("2.5-5", "SD")),
        (("0-2.5 vs 5-7.5", "NSD"), ("2.5-5", "SD")),
        (("0-2.5 vs 5-7.5", "NSD"), ("0-2.5 vs 5-7.5", "SD")),
    ]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw)
    annotator.apply_and_annotate()
    annotator.reset_configuration()

### Ripple and PBE rate

In [ ]:
y_labels = ["Ripple rate (Hz)", "PBE rate (Hz)"]
for d, (df, y) in enumerate(zip([rate, pbe_rate], ["rate", "rate"])):
    ax = fig.subplot(fig.gs[6, 2 * d : 2 * d + 2])
    plot_kw = dict(
        data=df,
        x="zt",
        y=y,
        hue="grp",
        hue_order=group,
        ax=ax,
        dodge=True,
    )

    sns.stripplot(
        **plot_kw, palette=subjects.colors_sd(1), edgecolor="w", linewidth=0.3, size=3
    )
    sns.barplot(
        **plot_kw,
        palette=subjects.colors_sd(1),
        ci=None,
        facecolor="w",
        linewidth=0.5,
        edgecolor=".2",
    )
    # violinplot(**plot_kw)
    ax.tick_params("x", labelrotation=30)
    ax.legend([], frameon=False)
    ax.set_ylabel(y_labels[d])
    ax.set_xlabel("")

    # stats
    # Across groups
    orders = df.zt.unique()
    pairs = [((_, "NSD"), (_, "SD")) for _ in orders] + [
        (("0-2.5", "NSD"), ("5-7.5", "SD"))
    ]
    annotator = Annotator(pairs=pairs, **plot_kw, order=orders)
    annotator.configure(test="t-test_welch", **stat_kw, color="#4AB33E")
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    # Within groups
    for i, g in enumerate(group):
        pairs2 = [((orders[_], g), (orders[_ + 1], g)) for _ in range(3)]
        annotator = Annotator(pairs=pairs2, **plot_kw, order=orders)
        annotator.configure(
            test="t-test_paired", **stat_kw, color=subjects.colors_sd(1)[i]
        )
        annotator.apply_and_annotate()
        annotator.reset_configuration()

### Firing inside/outside ripples

### ANOVA

In [ ]:
import pingouin as pg

rate["name2"] = [s[:4] for s in rate["name"]]

In [ ]:
from statsmodels.stats import anova

# rate =

st = anova.AnovaRM(
    data=pbe_rate[(pbe_rate.zt != "PRE")],
    depvar="rate",
    within=["zt", "grp"],
    subject="name2",
    aggregate_func=np.mean,
)
print(st.fit())

In [ ]:
pg.anova(
    data=rate[(rate.zt != "PRE")],
    dv="rate",
    between=["zt", "grp"],
    ss_type=3,
    detailed=True,
)

In [ ]:
for i, df in enumerate([rate, pbe_rate]):
    kw = dict(
        data=df[(df.zt != "PRE")],
        dv="rate",
        within="zt",
        between="grp",
        subject="session",
    )
    anv = pg.mixed_anova(**kw)
    pg.print_table(anv)
    posthocs = pg.pairwise_tests(**kw)
    pg.print_table(posthocs)

In [ ]:
# Let's assume that we have a balanced design with 30 students in each group
n = 30
months = ["August", "January", "June"]

# Generate random data
np.random.seed(1234)
control = np.random.normal(5.5, size=len(months) * n)
meditation = np.r_[
    np.random.normal(5.4, size=n),
    np.random.normal(5.8, size=n),
    np.random.normal(6.4, size=n),
]

# Create a dataframe
df = pd.DataFrame(
    {
        "Scores": np.r_[control, meditation],
        "Time": np.r_[np.repeat(months, n), np.repeat(months, n)],
        "Group": np.repeat(["Control", "Meditation"], len(months) * n),
        "Subject": np.r_[np.tile(np.arange(n), 3), np.tile(np.arange(n, n + n), 3)],
    }
)
# Compute the two-way mixed-design ANOVA
aov = pg.mixed_anova(
    dv="Scores", within="Time", between="Group", subject="Subject", data=df
)
# Pretty printing of ANOVA summary
pg.print_table(aov)

In [ ]:
df

In [ ]:
import pingouin as pg

for i, df in enumerate([rate, pbe_rate]):
    for g, grp in enumerate(["NSD", "SD"]):
        stat = pg.rm_anova(
            data=df[(df.zt != "PRE") & (df.grp == grp)],
            dv="rate",
            within=["zt", "grp"],
            subject="name2",
            detailed=True,
        )
        print(stat)

# pg.rm_anova(data=pbe_rate[(pbe_rate.zt!='PRE')&(pbe_rate.grp=='NSD')],dv='rate',within='zt',subject='session')
# pg.welch_anova(data=pbe_rate[pbe_rate.zt!='PRE'],dv='rate',between='grp')

### saving

In [ ]:
# fig.savefig(filename)